In [1]:
from sdup.pipelines.pipeline_stablediffusionupscale import StableVideoDiffusionUpscalePipeline
from sdup.models.unet import UNet3DConditionModel

import torch
from PIL import Image
from decord import VideoReader

from diffusers.utils import export_to_gif

/opt/conda/lib/python3.10/site-packages/diffusers/models/dual_transformer_2d.py:20: FutureWarning: `DualTransformer2DModel` is deprecated and will be removed in version 0.29. Importing `DualTransformer2DModel` from `diffusers.models.dual_transformer_2d` is deprecated and this will be removed in a future version. Please use `from diffusers.models.transformers.dual_transformer_2d import DualTransformer2DModel`, instead.
  deprecate("DualTransformer2DModel", "0.29", deprecation_message)


In [2]:
pipe_path = '/qiguojun/PreTrainedWeights/stable-diffusion-x4-upscaler'
unet = UNet3DConditionModel.from_pretrained_2d(pipe_path, subfolder='unet')
pipe = StableVideoDiffusionUpscalePipeline.from_pretrained(pipe_path, unet=unet).to('cuda')

missing_keys: <All keys matched successfully>


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

/qiguojun/PreTrainedWeights/stable-diffusion-x4-upscaler/text_encoder/model.safetensors
/qiguojun/PreTrainedWeights/stable-diffusion-x4-upscaler/text_encoder/model.safetensors


In [3]:
src_img = Image.open('./asset/sora.jpg').resize((128, 128))
prompt = 'a photo of five monster'

up_img = pipe(image=[src_img], prompt=prompt, num_inference_steps=25).images[0]
src_img.save('./src_img.jpg')
up_img.save('./up_img.jpg')

  0%|          | 0/25 [00:00<?, ?it/s]

In [4]:
vid_path = './asset/real01_inp.mp4'
frames_source = 8 # input video frames
video_reader = VideoReader(vid_path)
_, h, w, _ = video_reader.get_batch([0]).shape
video_ = video_reader.get_batch(range(len(video_reader))).asnumpy()[:frames_source]
video = []
for vid in video_:
    video.append(Image.fromarray(vid).resize((w//8, h//8)))
h = h//8; w = w//8
prompt='a photo of giraffes graze in the bushes'

video_up = pipe(prompt=prompt, image=video, num_inference_steps=25).images
export_to_gif(video, './src_video.gif')
export_to_gif(video_up, './up_video.gif')

  0%|          | 0/25 [00:00<?, ?it/s]

'./up_video.gif'